# Capstone Final Project -- Find Location for opening up a Halal Restaurant in Toronto

In [1]:
# import the dependencies all needed

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import json # library to handle JSON files
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
bs4-4.9.1 

## Scraping data from wikipidea page

In [86]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki,"html.parser")

table = soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')

my_data =[]

for row in table_rows:
    my_data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(my_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[1:]
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [87]:
# drop borough value that is not assigned
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

#Drop the the rows on which the Borough is empty
df.dropna(subset=['Borough'], inplace=True)

In [88]:
# change value in neighbourhood which is 'Not assigned' to borough value
def change_value(data, col_1, col_2):
    for index, row in data.iterrows():
        if row[col_1] == 'Not assigned':
            row[col_1] == row[col_2]

change_value(df,'Neighbourhood', 'Borough')

In [89]:
#Group by Postcode / Borough
df_post = df.groupby(['PostalCode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
df_post.reset_index(inplace=True)
df_post.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [90]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighbourhood"]
show_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    show_df = show_df.append(df_post[df_post["PostalCode"]==postcode], ignore_index=True)
    
show_df

,PostalCode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


### Load Geo coordinates data from csv file

In [91]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv', names=['PostalCode', 'Latitude', 'Longitude'], header=0)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge two datasets 

In [92]:
# merge two table on the column "PostalCode"
new_df = df_post.merge(geo_df, on="PostalCode", how="left")
new_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [93]:
# test
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
test_df = new_df[new_df['PostalCode'].isin(test_list)]
test_df.reset_index(drop=True)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M4G,East York,Leaside,43.709060,-79.363452
6,M4M,East Toronto,Studio District,43.659526,-79.340923
7,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


### Get boroguh name that only contains Toronto

In [94]:
df_toronto = new_df[new_df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [95]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighbourhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64


In [96]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

# average lat and lon of toronto area
lat_tor = df_toronto['Latitude'].mean()
lon_tor = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_tor, lon_tor))

The geographical coordinates of Toronto are 43.66713498717947, -79.38987324871795


In [97]:
# set random color for visualizing the markers
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) 
    

map_toronto = folium.Map(location=[lat_tor, lon_tor], zoom_start=12)

# map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [29]:
# save the map as HTML file
map_toronto.save('map_toronto.html')

### Define Foursquare Credentials and Version

In [98]:
CLIENT_ID = 'HG4SUHATWXJXWLABEVQB5WXMZGDWVZYUA2EAR0H5PMR4SSLN' # your Foursquare ID
CLIENT_SECRET = 'A3WMXPQQIREJQKJPHZCYJ4H2PG125QZWBA0LOGAOH1GNLPX4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HG4SUHATWXJXWLABEVQB5WXMZGDWVZYUA2EAR0H5PMR4SSLN
CLIENT_SECRET:A3WMXPQQIREJQKJPHZCYJ4H2PG125QZWBA0LOGAOH1GNLPX4


#### Obtain the venues from all neighbourhoods in Toronto

In [99]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [36]:
# toronto_venues.head()

In [101]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


#### Let'sfind out how many unique categories can be curated from all the returned venues

In [102]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


#### Check if those restaurant categoris are containec in all unique venues

In [103]:
"Middle Eastern Restaurant" in toronto_venues['Venue Category'].unique()
"Indian Restaurant" in toronto_venues['Venue Category'].unique()
"Mediterranean Restaurant" in toronto_venues['Venue Category'].unique()

True

In [104]:
temp1 = toronto_venues[toronto_venues['Venue Category'].str.contains('Indian Restaurant')]
temp2 = toronto_venues[toronto_venues['Venue Category'].str.contains('Middle Eastern Restaurant')]
temp3 = toronto_venues[toronto_venues['Venue Category'].str.contains('Mediterranean Restaurant')]

In [105]:
# helper function to print dataframes
from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [106]:
display_side_by_side([temp1, temp2, temp3], ['Indian', 'Middle Eastern', 'Mediterranean'])

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant
138,Davisville,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
198,"St. James Town, Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
268,Church and Wellesley,43.665860,-79.383160,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant
597,Berczy Park,43.644771,-79.373306,Bindia Indian Bistro,43.648559,-79.371816,Indian Restaurant
656,Central Bay Street,43.657952,-79.387383,Colaba Junction,43.660940,-79.385635,Indian Restaurant
852,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Indian Roti House,43.639060,-79.385422,Indian Restaurant
1075,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
1289,Stn A PO Boxes,43.646435,-79.374846,Bindia Indian Bistro,43.648559,-79.371816,Indian Restaurant
,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


#### Note: Replace Middle Eastern, Indian, Mediterranean restaurant with Halal Restaurant instead

In [107]:
# !!! Assume middle eastern, indian, and mediterranean food are Halal kind food in terms of food taste

toronto_venues.replace({'Venue Category' : {'Middle Eastern Restaurant' : 'Halal Restaurant', 
                                           'Indian Restaurant' : 'Halal Restaurant',
                                           'Mediterranean Restaurant' : 'Halal Restaurant'}}, inplace = True)

In [108]:
df_halal = toronto_venues[toronto_venues['Venue Category'].str.contains('Halal')]
df_halal

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Halal Restaurant
83,Studio District,43.659526,-79.340923,Tabule,43.659731,-79.346341,Halal Restaurant
138,Davisville,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Halal Restaurant
198,"St. James Town, Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Halal Restaurant
268,Church and Wellesley,43.665860,-79.383160,Kothur Indian Cuisine,43.667872,-79.385659,Halal Restaurant
296,Church and Wellesley,43.665860,-79.383160,The Salad House,43.669508,-79.386061,Halal Restaurant
299,Church and Wellesley,43.665860,-79.383160,Constantine,43.668773,-79.385287,Halal Restaurant
396,"Garden District, Ryerson",43.657162,-79.378937,Kabul Express,43.656691,-79.376643,Halal Restaurant
422,"Garden District, Ryerson",43.657162,-79.378937,Paramount Fine Foods,43.655029,-79.380245,Halal Restaurant
469,St. James Town,43.651494,-79.375418,Mystic Muffin,43.652484,-79.372655,Halal Restaurant


In [109]:
"Halal Restaurant" in toronto_venues['Venue Category'].unique()

True

## Analyze Each Neighborhood

In [110]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1642, 233)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Men's Store,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [111]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Men's Store,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.016667,0.0,0.0,0.016667,0.000000,0.0,0.0,0.0000,0.016667,0.0,0.016667,0.033333,0.0,0.000000,0.0,0.016667,0.016667,0.0,0.033333,0.0,0.0,0.000000,0.016667,0.0000,0.0,0.0000,0.0,0.016667,0.0000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.00,0.0,0.033333,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,0.000000,0.016667,0.00,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.016667,0.000000,0.016667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.016667,0.0,0.0,0.0,0.000000,0.033333,0.0000,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0.016667,0.016667,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0

Getting the top 10 for each neighbourhood

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Café,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Bakery,Breakfast Spot,Coffee Shop,Climbing Gym,Restaurant,Bar,Intersection,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Burrito Place,Park,Restaurant,Fast Food Restaurant,Garden Center,Garden,Gym / Fitness Center,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Plane,Harbor / Marina,Sculpture Garden,Airport Gate,Airport Food Court,Airport
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Bubble Tea Shop,Halal Restaurant,Salad Place,Department Store,Japanese Restaurant


In [114]:
# len(toronto_grouped[toronto_grouped["Halal Restaurant"] > 0])

#### Create dataframe only with Halal restaurants 

In [115]:
toronto_halal = toronto_grouped[['Neighborhood', 'Halal Restaurant']]
toronto_halal.head()

,Neighborhood,Halal Restaurant
0,Berczy Park,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.031746


## K-means for clustering neighbourhoods

In [116]:
# set number of clusters
toclusters = 3

to_clustering = toronto_halal.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 0, 2, 0, 2, 0, 2, 0], dtype=int32)

In [117]:
toronto_merged = toronto_halal.copy()

toronto_merged['Cluster Labels'] = kmeans.labels_
# toronto_merged.head()
toronto_merged = toronto_merged.join(toronto_venues.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

,Neighborhood,Halal Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.0,0,43.653206,-79.400049,Kanto,43.652167,-79.404843,Filipino Restaurant
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Vistek,43.657046,-79.359667,Electronics Store
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Arta Gallery,43.650022,-79.361222,Art Gallery
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Mill St. Brew Pub,43.650353,-79.358489,Pub
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Greg's Ice Cream,43.650102,-79.360440,Ice Cream Shop


### Visualize the clusters with folium

In [120]:
map_clusters = folium.Map(location=[lat_tor, lon_tor], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [80]:
# Cluster 0(First cluster)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Halal Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.000000,0,43.653206,-79.400049,Kanto,43.652167,-79.404843,Filipino Restaurant
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Cambridge Suites Toronto,43.651836,-79.378107,Hotel
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Bulldog On The Block,43.650652,-79.384141,Coffee Shop
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Versus Coffee,43.651213,-79.375236,Coffee Shop
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Over Easy,43.648077,-79.377599,Breakfast Spot
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Mercatto,43.650243,-79.380820,Italian Restaurant
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Portico,43.651856,-79.378055,Restaurant
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,John Allans,43.651447,-79.381354,Salon / Barbershop
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,Ruby Thai (First Canadian Place),43.649091,-79.381600,Thai Restaurant
7,"Commerce Court, Victoria Hotel",0.000000,0,43.648198,-79.379817,NAMI,43.650853,-79.375887,Japanese Restaurant


In [81]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Halal Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,TD Canada Trust,43.667934,-79.441698,Bank
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Nova Era Bakery,43.669886,-79.437582,Bakery
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,The Sovereign,43.673116,-79.440265,Café
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Parallel,43.669516,-79.438728,Halal Restaurant
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Rehearsal Factory,43.668877,-79.443603,Music Venue
10,"Dufferin, Dovercourt Village",0.071429,1,43.669005,-79.442259,Wallace Emerson Park,43.666933,-79.439449,Park
34,"The Annex, North Midtown, Yorkville",0.090909,1,43.672710,-79.405678,Martino's Pizza,43.675560,-79.403558,Pizza Place


In [82]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Halal Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,Davisville,0.028571,2,43.704324,-79.388790,souvlaki express,43.707378,-79.389848,Greek Restaurant
8,Davisville,0.028571,2,43.704324,-79.388790,Petro-Canada,43.702269,-79.387955,Gas Station
8,Davisville,0.028571,2,43.704324,-79.388790,Apple Tree Farmer's Market,43.700326,-79.389760,Farmers Market
13,"Garden District, Ryerson",0.020000,2,43.657162,-79.378937,The Senator Restaurant,43.655641,-79.379199,Diner
8,Davisville,0.028571,2,43.704324,-79.388790,Shoppers Drug Mart,43.707806,-79.389893,Pharmacy
8,Davisville,0.028571,2,43.704324,-79.388790,Subway,43.701763,-79.387805,Sandwich Place
8,Davisville,0.028571,2,43.704324,-79.388790,Pizza Pizza,43.706138,-79.389292,Pizza Place
8,Davisville,0.028571,2,43.704324,-79.388790,Second Cup,43.704344,-79.388659,Coffee Shop
8,Davisville,0.028571,2,43.704324,-79.388790,Subway,43.708474,-79.390674,Sandwich Place
8,Davisville,0.028571,2,43.704324,-79.388790,Kidnasium,43.706939,-79.389530,Indoor Play Area
